In [1]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType


In [2]:
spark = SparkSession.builder.appName("Apache PySpark Final Project-Zomato-Task").getOrCreate()

22/11/04 09:51:25 WARN Utils: Your hostname, HP-G62 resolves to a loopback address: 127.0.1.1; using 192.168.18.113 instead (on interface enp3s0)
22/11/04 09:51:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/04 09:51:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
cleaned_zomato = spark.read.csv('data/cleaned_zomato_df.csv',inferSchema=True,header=True)

In [4]:
cleaned_zomato.columns

['name',
 'online_order',
 'book_table',
 'rating',
 'votes',
 'phone',
 'location',
 'type',
 'dish_liked',
 'cuisines',
 'approx_cost_two_people']

# Task 0: convert "NEW" and "-" in rating to "0" and x/5 to x

In [5]:
def convert(strr):
    if(strr == 'NEW' or strr == '-'):
        return "0"
    else:
        value = str(strr).split('/')
        value = value[0]
        return float(value)


In [6]:
convertUDF = udf(lambda string : convert(string),StringType())

In [7]:
cleaned_zomato_rate = cleaned_zomato.withColumn("rating", convertUDF(col('rating')))
cleaned_zomato_rate.toPandas().to_csv('data/cleaned_zomato_rate.csv',index= False)
cleaned_zomato_rate.show()

+--------------------+------------+----------+------+-----+--------------+------------------+-------------------+--------------------+--------------------+----------------------+
|                name|online_order|book_table|rating|votes|         phone|          location|               type|          dish_liked|            cuisines|approx_cost_two_people|
+--------------------+------------+----------+------+-----+--------------+------------------+-------------------+--------------------+--------------------+----------------------+
|      Spice Elephant|         Yes|        No|   4.1|  787|  080 41714161|      Banashankari|      Casual Dining|Momos, Lunch Buff...|Chinese, North In...|                   800|
|     San Churro Cafe|         Yes|        No|   3.8|  918|+91 9663487993|      Banashankari|Cafe, Casual Dining|Churros, Cannello...|Cafe, Mexican, It...|                   800|
|Addhuri Udupi Bho...|          No|        No|   3.7|   88|+91 9620009302|      Banashankari|        Quic

In [8]:
cleaned_zomato_rating = spark.read.csv('data/cleaned_zomato_rate.csv',inferSchema=True,header=True)

In [9]:
cleaned_zomato_rating.show(5)

+--------------------+------------+----------+------+-----+--------------+------------+-------------------+--------------------+--------------------+----------------------+
|                name|online_order|book_table|rating|votes|         phone|    location|               type|          dish_liked|            cuisines|approx_cost_two_people|
+--------------------+------------+----------+------+-----+--------------+------------+-------------------+--------------------+--------------------+----------------------+
|      Spice Elephant|         Yes|        No|   4.1|  787|  080 41714161|Banashankari|      Casual Dining|Momos, Lunch Buff...|Chinese, North In...|                   800|
|     San Churro Cafe|         Yes|        No|   3.8|  918|+91 9663487993|Banashankari|Cafe, Casual Dining|Churros, Cannello...|Cafe, Mexican, It...|                   800|
|Addhuri Udupi Bho...|          No|        No|   3.7|   88|+91 9620009302|Banashankari|        Quick Bites|         Masala Dosa|South I

In [10]:
cleaned_zomato_rating.columns

['name',
 'online_order',
 'book_table',
 'rating',
 'votes',
 'phone',
 'location',
 'type',
 'dish_liked',
 'cuisines',
 'approx_cost_two_people']

# Task 1: Find resturants which can provide Italian items.

In [11]:
from pyspark.sql.functions import split, col
cuisine_list = cleaned_zomato_rating\
    .select(col('name'),col('location'),col('cuisines'),split(col("cuisines"),",").alias("cuisines_list"))\
    .dropDuplicates()

cuisine_list.show()

+--------------------+--------------------+--------------------+--------------------+
|                name|            location|            cuisines|       cuisines_list|
+--------------------+--------------------+--------------------+--------------------+
|           Java City|           Jayanagar|Cafe, American, P...|[Cafe,  American,...|
|Corner House Ice ...|           Jayanagar| Ice Cream, Desserts|[Ice Cream,  Dess...|
|                 KFC|   Bannerghatta Road|   Burger, Fast Food|[Burger,  Fast Food]|
|   Sreeraj Lassi Bar|        Shanti Nagar| Beverages, Desserts|[Beverages,  Dess...|
|               Petoo|                 HSR|North Indian, Fas...|[North Indian,  F...|
|            Foodbell|       Sarjapur Road|North Indian, Chi...|[North Indian,  C...|
|Donne Biriyani House|         Indiranagar|             Biryani|           [Biryani]|
|Polo Club - The O...|             MG Road|American, North I...|[American,  North...|
| The Spice Pavillion|        Marathahalli|        Nor

In [12]:
italian_items=cleaned_zomato_rating.filter(col('cuisines').like("%Italian%").alias("Italian Item"))\
    .orderBy(col("votes").desc(),col("rating").desc())

italian_items.toPandas().to_csv('data/italian_items.csv',index = False)
italian_items.show()

+--------------------+------------+----------+------+-----+--------------+-------------+-------------------+--------------------+--------------------+----------------------+
|                name|online_order|book_table|rating|votes|         phone|     location|               type|          dish_liked|            cuisines|approx_cost_two_people|
+--------------------+------------+----------+------+-----+--------------+-------------+-------------------+--------------------+--------------------+----------------------+
|Byg Brewski Brewi...|         Yes|       Yes|   4.9|16832|+91 8039514766|Sarjapur Road|       Microbrewery|Cocktails, Butter...|Continental, Nort...|                 1,600|
|Byg Brewski Brewi...|         Yes|       Yes|   4.9|16832|+91 8039514766|Sarjapur Road|       Microbrewery|Cocktails, Butter...|Continental, Nort...|                 1,600|
|Byg Brewski Brewi...|         Yes|       Yes|   4.9|16832|+91 8039514766|Sarjapur Road|       Microbrewery|Cocktails, Butter...|C

In [13]:
dish_liked=cleaned_zomato_rating.filter(col('dish_liked').like("%Mushroom Ravioli%").alias("Mushroom Ravioli"))

dish_liked.toPandas().to_csv('data/dish_liked.csv',index = False)
dish_liked.show()


+------+------------+----------+------+-----+--------------+------------+-------------------+--------------------+--------------------+----------------------+
|  name|online_order|book_table|rating|votes|         phone|    location|               type|          dish_liked|            cuisines|approx_cost_two_people|
+------+------------+----------+------+-----+--------------+------------+-------------------+--------------------+--------------------+----------------------+
|Onesta|         Yes|       Yes|   4.6| 1755|+91 8049525686|Basavanagudi|Casual Dining, Cafe|Barbeque Chicken ...|Pizza, Cafe, Italian|                   600|
|Onesta|         Yes|       Yes|   4.6| 1755|+91 8049525686|Basavanagudi|Casual Dining, Cafe|Barbeque Chicken ...|Pizza, Cafe, Italian|                   600|
|Onesta|         Yes|       Yes|   4.4| 3738|+91 7337844620|         HSR|Casual Dining, Cafe|Banana Smoothie, ...|Pizza, Cafe, Italian|                   600|
|Onesta|         Yes|       Yes|   4.4| 3764|+

# Task 2: Most liked restaurant in the city Banashankari

In [14]:
location_Banashankari=cleaned_zomato_rating\
    .filter(col("location")== "Banashankari")\
    .orderBy(col("votes").desc(),col("rating").desc())

location_Banashankari.toPandas().to_csv('data/location_Banashankari.csv',index= False)

location_Banashankari.show(10,truncate=False)

+---------------+------------+----------+------+-----+--------------+------------+-------------------+-----------------------------------------------------------------------------------------------+-------------------------------------+----------------------+
|name           |online_order|book_table|rating|votes|phone         |location    |type               |dish_liked                                                                                     |cuisines                             |approx_cost_two_people|
+---------------+------------+----------+------+-----+--------------+------------+-------------------+-----------------------------------------------------------------------------------------------+-------------------------------------+----------------------+
|Vinny's        |Yes         |Yes       |4.2   |2130 |080 26716795  |Banashankari|Casual Dining      |Pizza, Pasta, Garlic Bread, Nachos, Penne Alfredo, Mozzarella Sticks, Salad                    |Pizza, Italian, Contin

# Task 3: Change Yes to True and No to False in columns online_order using UDF.

In [15]:
def online_order_and_book_table(strr):
    if strr == "Yes":
        return True
    
    elif strr == "No":
        return False
    
    else:
        return False

In [16]:
convertUDF = udf(lambda string : online_order_and_book_table(string),StringType())

In [18]:
online_order = cleaned_zomato_rating.withColumn("online_order", convertUDF(col('online_order')))
cleaned_zomato_rating=online_order
online_order.toPandas().to_csv('data/online_order.csv',index= False)
online_order.show()

+--------------------+------------+----------+------+-----+--------------+------------------+-------------------+--------------------+--------------------+----------------------+
|                name|online_order|book_table|rating|votes|         phone|          location|               type|          dish_liked|            cuisines|approx_cost_two_people|
+--------------------+------------+----------+------+-----+--------------+------------------+-------------------+--------------------+--------------------+----------------------+
|      Spice Elephant|        true|        No|   4.1|  787|  080 41714161|      Banashankari|      Casual Dining|Momos, Lunch Buff...|Chinese, North In...|                   800|
|     San Churro Cafe|        true|        No|   3.8|  918|+91 9663487993|      Banashankari|Cafe, Casual Dining|Churros, Cannello...|Cafe, Mexican, It...|                   800|
|Addhuri Udupi Bho...|       false|        No|   3.7|   88|+91 9620009302|      Banashankari|        Quic

# Task 4: Change Yes to True and No to False in columns  book_table using UDF.

In [19]:
convertUDF = udf(lambda string : online_order_and_book_table(string),StringType())

In [20]:
book_table = cleaned_zomato_rating.withColumn("book_table", convertUDF(col('book_table')))
cleaned_zomato_rating=book_table
book_table.toPandas().to_csv('data/book_table.csv',index= False)
book_table.show()

+--------------------+------------+----------+------+-----+--------------+------------------+-------------------+--------------------+--------------------+----------------------+
|                name|online_order|book_table|rating|votes|         phone|          location|               type|          dish_liked|            cuisines|approx_cost_two_people|
+--------------------+------------+----------+------+-----+--------------+------------------+-------------------+--------------------+--------------------+----------------------+
|      Spice Elephant|        true|     false|   4.1|  787|  080 41714161|      Banashankari|      Casual Dining|Momos, Lunch Buff...|Chinese, North In...|                   800|
|     San Churro Cafe|        true|     false|   3.8|  918|+91 9663487993|      Banashankari|Cafe, Casual Dining|Churros, Cannello...|Cafe, Mexican, It...|                   800|
|Addhuri Udupi Bho...|       false|     false|   3.7|   88|+91 9620009302|      Banashankari|        Quic

# Task 5: List the best restaurants in each location(best in accordance with rate)



In [21]:
best_restaurants=cleaned_zomato_rating.groupBy("name","location")
best_restaurants.max().show()

+--------------------+--------------------+-----------+----------+
|                name|            location|max(rating)|max(votes)|
+--------------------+--------------------+-----------+----------+
|Belly Squad Food ...|     South Bangalore|        3.8|        55|
|         Bakers Town|                 BTM|        3.6|        42|
|            Foodbell|       Sarjapur Road|        0.0|         0|
|             Zeeshan|       Sarjapur Road|        2.6|       156|
|             Chatora|        Marathahalli|        3.5|       115|
|      Drunken Monkey|        Marathahalli|        3.9|        98|
|    Banana Beach Bar|Koramangala 6th B...|        3.7|      1016|
|    House Of Commons|Koramangala 5th B...|        4.8|      4470|
|          Chai Point|     Cunningham Road|        3.8|       149|
|       Aligarh House|                 HSR|        4.4|       568|
|  Mani's Dum Biryani|  Jeevan Bhima Nagar|        4.0|       691|
|           Just Bake|  Jeevan Bhima Nagar|        3.0|       

# Task 6:Find the total no. of voters in each location (window function)

In [22]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,udf,col,sum
windowSpec = Window.partitionBy('location')

total_votes_by_city = cleaned_zomato_rating.withColumn('total_votes',sum(col('votes')).over(windowSpec))\
                                .select(col('location'),col('total_votes')).dropDuplicates()

total_votes_by_city.toPandas().to_csv('data/total_votes_by_location.csv',index = False)

total_votes_by_city.show()

+------------------+-----------+
|          location|total_votes|
+------------------+-----------+
|               BTM|     248350|
|      Banashankari|      53715|
|         Banaswadi|      12870|
| Bannerghatta Road|     119575|
|      Basavanagudi|      55228|
|Basaveshwara Nagar|       7123|
|         Bellandur|      82887|
|      Bommanahalli|        602|
|      Brigade Road|     196476|
|       Brookefield|      46651|
|    CV Raman Nagar|       1363|
| Central Bangalore|       1150|
|     Church Street|     261087|
|       City Market|        488|
| Commercial Street|      12276|
|   Cunningham Road|      39254|
|            Domlur|      10604|
|    East Bangalore|       2231|
|           Ejipura|      10400|
|   Electronic City|      38235|
+------------------+-----------+
only showing top 20 rows



# Task 7: Find the list of restaurant based on the location and  and rating 

In [23]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
windowSpec  = Window.partitionBy("location").orderBy(col("rating").desc())
list_restaurants=cleaned_zomato_rating.withColumn("row_number",row_number().over(windowSpec))
list_restaurants.toPandas().to_csv('data/list_restaurants.csv',index= False)
list_restaurants.show(10)

+--------------------+------------+----------+------+-----+--------------+--------+--------------+--------------------+--------------------+----------------------+----------+
|                name|online_order|book_table|rating|votes|         phone|location|          type|          dish_liked|            cuisines|approx_cost_two_people|row_number|
+--------------------+------------+----------+------+-----+--------------+--------+--------------+--------------------+--------------------+----------------------+----------+
|AB's - Absolute B...|       false|      true|   4.9| 6375|  040 45659912|     BTM| Casual Dining|Tangdi Chicken, B...|European, Mediter...|                 1,600|         1|
|AB's - Absolute B...|       false|      true|   4.9| 6375|  040 45659912|     BTM| Casual Dining|Tangdi Chicken, B...|European, Mediter...|                 1,600|         2|
|AB's - Absolute B...|       false|      true|   4.9| 6404|  040 45659912|     BTM| Casual Dining|Tangdi Chicken, B...|Europe